In [1]:
# Point source matching script between heritage programme and 2MASS infrared source catalogue
# This script creats a source list (cross_match_xmm_2mass.csv) of matched object between XMM and 2MASS
# observations
# AUTHOR : Samaresh Mondal (smondal@camk.edu.pl)
# DATE   : 2021 Sep 13

In [2]:
import numpy as np
import math
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy.io import fits
from astropy.table import Table
import re
from astroquery.vizier import Vizier

In [3]:
# XMM count rate to flux conversion using nH=1e22, powelaw index=2.0 and normaization=2.212E-03
pn_ctr_to_flux = 5.111E-12  # 0.3-10 keV band flux from 0.2-12 keV band count rate for PN
m_ctr_to_flux = 1.461E-11   # 0.3-10 keV band flux from 0.2-12 keV band count rate for MOS

In [4]:
# Observation ids of heritage pointings
obsids_heritage=[['1','0886010101'],['2','0886010401'],['3','0886010501'],
                 ['4','0886010601'],['5','0886010301'],['6','0886010701'],
                 ['7','0886010801'],['8','0886010201'],['9','0886010901'],
                 ['10','0886011001'],['11','0886011101'],['12','0886011201'],
                 ['13','0886020301'],['14','0886020101'],['15','0886020301'],
                 ['16','0886011301']]

In [5]:
valuesep = 15.0 # Seperation value to consider good match. You may change this value
fp1 = open("cross_match_xmm_2mass.csv","w")
fp1.write("   %s        %s       %s     %s  %s %s %s   %s %s\n"%("RA","DEC","XMM-ID","RA(nvss)",\
                                "DEC(nvss)","XMM-flux(0.3-10keV)","Jmag","Kmag","HR(12-1keV/0.2-1keV)"))
for i in range(len(obsids_heritage)):
    #print(i)
    # Reading the xmm source list
    index = i
    obsid = obsids_heritage[index][1]
    xmm = fits.getdata('M1S001M2S002PNS003_'+str(obsid)+'_emllist_formatted.fits', 1)
    txmm = Table(xmm)

    ra_xmm = txmm['RA']                           # in degree
    dec_xmm = txmm['DEC']                         # in degree
    pos_err_xmm = txmm['RADEC_ERR']               # in arcsec
    srcnumber_xmm = txmm['ML_ID_SRC']
    pos_totalerr_xmm = txmm['TOTAL_RADEC_ERR']
 
    # EPIC-PN count rates
    pn_ctr = txmm['RATE_pn']                      # 0.2-12.0 keV
    pn_ctr1 = txmm['RATE_pn_1']                   # 0.2-0.5 keV
    pn_ctr2 = txmm['RATE_pn_2']                   # 0.5-1.0 keV
    pn_ctr3 = txmm['RATE_pn_3']                   # 1.0-2.0 keV
    pn_ctr4 = txmm['RATE_pn_4']                   # 2.0-4.5 keV
    pn_ctr5 = txmm['RATE_pn_5']                   # 4.5-12.0 keV
    pn_ctr6 = pn_ctr1+pn_ctr2+pn_ctr3+pn_ctr4     # 0.2-4.5 keV band
    pn_HR = (pn_ctr3+pn_ctr4+pn_ctr5)/(pn_ctr1+pn_ctr2) # 1-12 keV/0.2-1 keV
    
    pn_ctr_err = txmm['RATE_ERR_pn']
    pn_ctr_err1 = txmm['RATE_ERR_pn_1']
    pn_ctr_err2 = txmm['RATE_ERR_pn_2']
    pn_ctr_err3 = txmm['RATE_ERR_pn_3']
    pn_ctr_err4 = txmm['RATE_ERR_pn_4']
    pn_ctr_err5 = txmm['RATE_ERR_pn_5']
    pn_ctr_err6 = np.sqrt(pow(pn_ctr_err1,2.0)+pow(pn_ctr_err2,2.0)+pow(pn_ctr_err3,2.0)+pow(pn_ctr_err4,2.0))
    #HR_err = 
    
    # EPIC-MOS1 count rates
    m1_ctr = txmm['RATE_m1']                      # 0.2-12.0 keV
    m1_ctr1 = txmm['RATE_m1_1']                   # 0.2-0.5 keV
    m1_ctr2 = txmm['RATE_m1_2']                   # 0.5-1.0 keV
    m1_ctr3 = txmm['RATE_m1_3']                   # 1.0-2.0 keV
    m1_ctr4 = txmm['RATE_m1_4']                   # 2.0-4.5 keV
    m1_ctr5 = txmm['RATE_m1_5']                   # 4.5-12.0 keV
    m1_ctr6 = pn_ctr1+pn_ctr2+pn_ctr3+pn_ctr4     # 0.2-4.5 keV band
    m1_HR = (pn_ctr3+pn_ctr4+pn_ctr5)/(pn_ctr1+pn_ctr2) # 1-12 keV/0.2-1 keV
    
    # EPIC-MOS2 count rates
    m2_ctr = txmm['RATE_m1']                      # 0.2-12.0 keV
    m2_ctr1 = txmm['RATE_m1_1']                   # 0.2-0.5 keV
    m2_ctr2 = txmm['RATE_m1_2']                   # 0.5-1.0 keV
    m2_ctr3 = txmm['RATE_m1_3']                   # 1.0-2.0 keV
    m2_ctr4 = txmm['RATE_m1_4']                   # 2.0-4.5 keV
    m2_ctr5 = txmm['RATE_m1_5']                   # 4.5-12.0 keV
    m2_ctr6 = pn_ctr1+pn_ctr2+pn_ctr3+pn_ctr4     # 0.2-4.5 keV band
    m2_HR = (pn_ctr3+pn_ctr4+pn_ctr5)/(pn_ctr1+pn_ctr2) # 1-12 keV/0.2-1 keV
    
    # Change the coordinate RA and DEC into a zip
    coo_xmm = SkyCoord(ra_xmm*u.degree, dec_xmm*u.degree)
    
    maxrad = 0.6 * u.degree                          # Search radius of 2MASS sources
    result = Vizier(row_limit = -1,columns=['*','Date']).query_region(coo_xmm[0], radius=maxrad, catalog="II/246")
    tmass_table = result[0]
    coo_2mass = SkyCoord(tmass_table['RAJ2000'],tmass_table['DEJ2000'])
    
    # Cross match between XMM and 2MASS source catalogue
    idx1, d2d1, d3d1 = coo_xmm.match_to_catalog_sky(coo_2mass)
    
    # Define max sep in arcsec to consider good match
    max_sep = valuesep * u.arcsec
    # Apply mask true or false according to seperation limit 
    sep_cons1 = d2d1 < max_sep
    
    coo_xmm_matches1 = coo_xmm[sep_cons1]
    
    ra_xmm1 = ra_xmm[sep_cons1]
    dec_xmm1 = dec_xmm[sep_cons1]
    pn_flux1 = pn_ctr[sep_cons1]*pn_ctr_to_flux
    m1_flux1 = m1_ctr[sep_cons1]*m_ctr_to_flux
    m2_flux1 = m2_ctr[sep_cons1]*m_ctr_to_flux
    HR_pn = pn_HR[sep_cons1]
    HR_m1 = m1_HR[sep_cons1] 
    HR_m2 = m2_HR[sep_cons1]
    ra_2mass1 = tmass_table['RAJ2000'][idx1[sep_cons1]]
    dec_2mass1 = tmass_table['DEJ2000'][idx1[sep_cons1]]
    Jmag1 = tmass_table['Jmag'][idx1[sep_cons1]]
    Kmag1 = tmass_table['Kmag'][idx1[sep_cons1]]
    
    for k in range(len(ra_xmm1)):
        xmmflux = pn_flux1[k]
        HR = HR_pn[k]
        if(math.isnan(xmmflux)==True or math.isnan(HR)==True):
            xmmflux = m1_flux1[k]
            HR = HR_m1[k]
        if(math.isnan(xmmflux)==True or math.isnan(HR)==True):
            xmmflux = m2_flux1[k]
            HR = HR_m2[k]
        fp1.write("%f %f %d %f %f   %g      %g %g %g\n"%(ra_xmm1[k],dec_xmm1[k],int(obsid),ra_2mass1[k],\
                                                         dec_2mass1[k],xmmflux,Jmag1[k],Kmag1[k],HR))
        
fp1.close()

/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered 

/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
